In [106]:
import cv2
import numpy as np
import os

import ipywidgets as widgets
from matplotlib import pyplot as plt
from IPython.display import display

In [58]:
def clipImport(filenumber):
    filestr = str(filenumber)
    video = cv2.VideoCapture("clips/"+filestr)
    return video

In [91]:
cap = cv2.VideoCapture('clips/0.mp4')
 
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.mp4',fourcc, 24, (512,512))
 
while True:
    ret, frame = cap.read()
    if ret == True:
        b = cv2.resize(frame,(512,512),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
        out.write(b)
    else:
        break
    
cap.release()
out.release()
cv2.destroyAllWindows()

In [94]:
cap = cv2.VideoCapture('clips/1.mp4')

# (x, y, w, h) = cv2.boundingRect(c)
# cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 255, 0), 20)
# roi = frame[y:y+h, x:x+w]

while True:
    ret, frame = cap.read()
    sky = frame[104:616, 384:896]
    cv2.imwrite('Video', sky)

    if cv2.waitKey(1) == 27:
        exit(0)

TypeError: 'NoneType' object is not subscriptable

In [84]:
def splitFrames(video):
    ret, frame = video.read()
    path = ("clips/frames/frame_{}")
    counter = 0 
    print(path.format(counter))
    while ret:
        ret, frame = video.read()
        if not ret:
            break
        cv2.imwrite(path.format(counter), frame)
        counter += 1
    video.release()
    
film1 = cv2.VideoCapture("clips/3.mp4")
splitFrames(film1)

clips/frames/frame_0


error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:661: error: (-2:Unspecified error) could not find a writer for the specified extension in function 'cv::imwrite_'


In [81]:
videoDict = {}
for i in range(24):
    videoDict["video_{0}".format(i)] = clipImport(i)
    splitFrames(videoDict["video_{0}".format(i)])
print(videoDict)

{'video_0': <VideoCapture 0000019CC34EC4F0>, 'video_1': <VideoCapture 0000019CC34EC2D0>, 'video_2': <VideoCapture 0000019CC34EC0B0>, 'video_3': <VideoCapture 0000019CC34EC350>, 'video_4': <VideoCapture 0000019CC34EC3D0>, 'video_5': <VideoCapture 0000019CC34EC070>, 'video_6': <VideoCapture 0000019CC34EC250>, 'video_7': <VideoCapture 0000019CC34EC450>, 'video_8': <VideoCapture 0000019CC59F28D0>, 'video_9': <VideoCapture 0000019CC59F2870>, 'video_10': <VideoCapture 0000019CC59F25F0>, 'video_11': <VideoCapture 0000019CC59F27D0>, 'video_12': <VideoCapture 0000019CC59F2730>, 'video_13': <VideoCapture 0000019CC59F2670>, 'video_14': <VideoCapture 0000019CC59F26D0>, 'video_15': <VideoCapture 0000019CC59F2690>, 'video_16': <VideoCapture 0000019CC59F2550>, 'video_17': <VideoCapture 0000019CC7212DF0>, 'video_18': <VideoCapture 0000019CC7212930>, 'video_19': <VideoCapture 0000019CC72122F0>, 'video_20': <VideoCapture 0000019CC7212690>, 'video_21': <VideoCapture 0000019CC72129B0>, 'video_22': <VideoC

In [83]:
film = cv2.VideoCapture("clips/2.mp4")
disp = widgets.Image()
display(disp)

ret, frame = film.read()
while ret:
    ret, frame = film.read()
    if not ret:
        continue

    display_image = cv2.imencode('.png', frame)[1].tostring()
    disp.value = display_image
video_reader.release()
#splitFrames(film)

Image(value=b'')

In [109]:


def noisy(noise_typ,image):
    if noise_typ == "gauss":
        row,col,ch= image.shape
        mean = 0
        var = 0.1
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col,ch))
        gauss = gauss.reshape(row,col,ch)
        noisy = image + gauss
        return noisy
    elif noise_typ == "s&p":
        row,col,ch = image.shape
        s_vs_p = 0.5
        amount = 0.004
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))
                  for i in image.shape]
        out[coords] = 1

      # Pepper mode
        num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))
                  for i in image.shape]
        out[coords] = 0
        return out
    elif noise_typ == "poisson":
        vals = len(np.unique(image))
        vals = 2 ** np.ceil(np.log2(vals))
        noisy = np.random.poisson(image * vals) / float(vals)
        return noisy
    elif noise_typ =="speckle":
        row,col,ch = image.shape
        gauss = np.random.randn(row,col,ch)
        gauss = gauss.reshape(row,col,ch)        
        noisy = image + image * gauss
        return noisy

In [105]:
output_filename = "processed_clips/1/processed_1.mp4"
noised_output_filename = "processed_clips/1/noised_1.mp4"
backend = cv2.CAP_ANY
fourcc_code = cv2.VideoWriter_fourcc(*"VP80")
fps = 24
frame_size = (512, 512)
output_video = cv2.VideoWriter(output_filename, backend, fourcc_code, fps, frame_size)
noised_output_video = cv2.VideoWriter(noised_output_filename, backend, fourcc_code, fps, frame_size)

input_video = cv2.VideoCapture("clips/1.mp4")
ret, frame = input_video.read()
path = "processed_clips/1/frame_{}.png"
print(type(frame))
counter = 0
print(path.format(counter))
while ret:
    ret, frame = input_video.read()
    
    if not ret:
        break
    cropped = frame[104:616, 384:896]
    #location = path.format(counter)
    cv2.imwrite(path.format(counter), cropped)
    output_video.write(cropped)
    counter += 1
print(type(cropped))
input_video.release()
output_video.release()

<class 'numpy.ndarray'>
processed_clips/1/frame_0.png
<class 'numpy.ndarray'>
